In [ ]:
#Creating a subset of the data

#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "/Users/sabihabarlaskar/Desktop/udacity-nanodegree/P3 Data wrangling/project/austin_texas.osm"  # Replace this with your osm file
SAMPLE_FILE = "/Users/sabihabarlaskar/Desktop/udacity-nanodegree/P3 Data wrangling/project/austin.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [ ]:
#tags.py

#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.attrib['k']):
            keys['lower'] +=1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon']+=1
        elif problemchars.search(element.attrib['k']):
            keys['problemchars']+=1
        else:
            keys['other']+=1
        
        pass
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


process_map('/Users/sabihabarlaskar/Desktop/udacity-nanodegree/P3 Data wrangling/project/austin.osm')

In [ ]:
#users.py

import xml.etree.cElementTree as ET
import pprint
import re


def get_user(element):
    user = element.attrib['user']
    return user


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.tag == "node":
            users.add(get_user(element))
        pass

    return users


def test():

    users = process_map('/Users/sabihabarlaskar/Desktop/udacity-nanodegree/P3 Data wrangling/project/austin.osm')
    pprint.pprint(users)
    assert len(users) == 6



if __name__ == "__main__":
    test()

In [ ]:
#mapparser.py
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict

def count_tags(filename):
    counts = defaultdict(int)
    for line in ET.iterparse(filename):
        current = line[1].tag
        counts[current] += 1
    return counts
def test():

    tags = count_tags('/Users/sabihabarlaskar/Desktop/udacity-nanodegree/P3 Data wrangling/project/austin.osm')
    pprint.pprint(tags)
    
if __name__ == "__main__":
    test()


In [ ]:
#audit
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "/Users/sabihabarlaskar/Desktop/udacity-nanodegree/P3 Data wrangling/project/austin.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
zipcodes = set()
phone_num = []

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Dr" : "Drive",
            "Pkwy" : "Parkway",
            "Ln" : "Lane",
            "CV" : "Cove",
            "Cv" : "Cove",
            "Hwy" : "Highway",
            "Expy" : "Expressway",
            "Ave": "Avenue",
            "Rd": "Road",
            "W.": "West",
            "N.": "North",
            "S.": "South",
            "E": "East",
            "Blvd." : "Boulevard",
            "Blvd" : "Boulevard"}

##Checking the different street types

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
    return street_types
 
##Checking if the zipcode conforms to the total lenght of 5    
def audit_zipcode(zipcodes, zipcode):
    
    if re.match(r'^\d{5}$', zipcode):
        zipcodes.add(zipcode)
    if re.match(r'^\d{5}\:\d{5}$', zipcode):
        zips = zipcode.split(":")
        zipcodes.add(zips[0])
    
    return zipcodes


def audit_phone(phone_num, phone):
    pattern = r'^\(?([0-9]{3})\)?[-.●]?([0-9]{3})[-.●]?([0-9]{4})$'
    if re.match(r'^\(?[+1]?([0-1]{1})\)?[ -]?([0-9]{3})?[-.●  ]?([0-9]{3})[-.●  ]?([0-9]{4})$', phone):
        phone_num.append(phone)
    if re.match(r'^\(?([0-9]{3})\)?[-.●]?([0-9]{3})[-.●]?([0-9]{4})$', phone):
        phone = "+1 " + phone
        phone_num.append(phone)
        return phone_num

def is_phone_num(elem):
    return 'phone' in elem.attrib['k'] 

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def is_zip_code(elem):
    return 'zip' in elem.attrib['k']

def audit(osmfile):
    osm_file = open(osmfile, "r")
    
    street_types = defaultdict(set)
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                if is_zip_code(tag):
                    audit_zipcode(zipcodes, tag.attrib['v'])
                if is_phone_num(tag):
                    audit_phone(phone_num, tag.attrib['v'])
                #print phone_num
    return street_types, zipcodes, phone_num

                
    #return {'Street': street_types,'Zip':zipcodes, 'Phone numbers':phone_num}
                    

def update_name(name, mapping):
    after = []
    # Split name string to test each part of the name;
    # Replacements may come anywhere in the name.
    for part in name.split(" "):
        # Check each part of the name against the keys in the correction dict
        if part in mapping.keys():
            # If exists in dict, overwrite that part of the name with the dict value for it.
            part = mapping[part]
        # Assemble each corrected piece of the name back together.
        after.append(part)
    # Return all pieces of the name as a string joined by a space.
    return " ".join(after)
    return name

def audit_test():
    st_types,zips,ph = audit(OSMFILE)
    
    #assert len(st_types) == 3
    #pprint.pprint(dict(st_types))
    #print(zips)
    #pprint.pprint(st_types) 
    updated_phones = update_phone(ph)
    #print updated_phones
    #print zips
        
    
    for st_type, ways in st_types.iteritems():
    
        #print st_type,
        print ways
        
        for name in ways:
         
                
            update_name(name, mapping)
            better_name = update_name(name, mapping)
            #print better_name
            #print name, "=>", better_name
            
    


if __name__ == '__main__':
    audit_test()

In [ ]:
#data.py
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
import audit


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
mapping = { "St": "Street",
            "St.": "Street",
            "Dr" : "Drive",
            "Pkwy" : "Parkway",
            "Ln" : "Lane",
            "CV" : "Cove",
            "Cv" : "Cove",
            "Hwy" : "Highway",
            "HWY" : "Highway",
            "Expy" : "Expressway",
            "Ave": "Avenue",
            "Rd": "Road",
            "W.": "West",
            "N.": "North",
            "S.": "South",
            "E": "East",
            "Blvd." : "Boulevard",
            "Blvd" : "Boulevard"}

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
ZIPCODE_TAGS = ['addr:postcode', 'tiger:zip_left', 'tiger:zip_left_1', 'tiger:zip_left_2',
    'tiger:zip_left_3', 'tiger:zip_left_4', 'tiger:zip_right', 'tiger:zip_right_1',
    'tiger:zip_right_2', 'tiger:zip_right_3', 'tiger:zip_right_4']
phone_number_key = ["contact:phone","phone"]
IGNORED_TAGS = ['gnis:ST_num', 'text', 'tiger:Name', 'gnis:id', 'is_in',
    'gnis:feature_type', 'lake:surface_area:acres', 'gnis:county_id', 'iata',
    'stop', 'trees', 'icao', 'gnis:County', 'gnis:county_num', 'name:en', 'gnis:state_id',
    'health_specialty:palliative_medicine', 'tiger:STATEFP', 'name:ru', 'name:uk'
    'wikipedia:en', 'Hardware Store', 'isced', 'reg_ref', 'start_date',
    'reg_name', 'al', 'isced:level', 'source:maxspeed', 'gnis_state_id',
    'undefined', 'int_ref', 'source:ref:note', 'gnis:ST_alpha', 'gnis:feature_id',
    'practice', 'lake:shore_length:miles', 'gnis:edited', 'gnis:freature_id',
    'name:ar', 'cycleway:left', 'import_uuid', 'odbl:note', 'is_in:state',
    'gnis:reviewed', 'name:backward', 'gnis:fcode', 'is_in:country_code',
    'is_in:iso_3166_2', 'name:brand', 'name:pl', 'gnis:st_alpha']

zipcodes = []
phone_num = []
phone_numbers = []
node = {}
address_info = {}
nd_info = []


def is_address(elem):
    if elem.attrib['k'][:5] == "addr:":
        return True

def shape_element(element):
    
    if element.tag == "node" or element.tag == "way" :
        #pprint.pprint(element.attrib)
        node["type"] = element.tag
        node["id"] = element.attrib["id"]
        if "visible" in element.attrib.keys():
            node["visible"] = element.attrib["visible"]
        if "lat" in element.attrib.keys():
            node["pos"] = [float(element.attrib['lat']), float(element.attrib['lon'])]
        node["created"] = {"version": element.attrib['version'],
                            "changeset": element.attrib['changeset'],
                            "timestamp": element.attrib['timestamp'],
                            "uid": element.attrib['uid'],
                            "user": element.attrib['user']}
        for tag in element.iter("tag"):
            p = problemchars.search(tag.attrib['k'])
            ##If problematic characters, skip it
            if p:
                #print "PROBLEM:", p.group()
                continue
            #Cleaning and extracting the street names, housenumber and zipcodes
            if tag.attrib['k'][:5] == "addr:":
                if ":" in tag.attrib['k'][5:]:
                    continue
                else:
                    
                    if tag.attrib['k'] == "addr:street":
                        string = tag.attrib['v']
                        groups = [group.strip() for group in string.split(';')]
                        street_names = audit.update_name(groups[0],mapping)
                        address_info['Street'] = street_names
                    if tag.attrib['k'] == "addr:housenumber":
                        address_info["housenumber"] = tag.attrib['v']
                        
            
                        #print address_info
            if tag.attrib['k'] in ZIPCODE_TAGS:
                clean_zipcode(zipcodes, tag.attrib['v'])
                for zips in zipcodes:
                    address_info['zip'] = zips
                #print address_info
            #Adding phone numbers
            if tag.attrib['k'] in phone_number_key:
                phone = clean_phonenumber(phone_num,tag.attrib['v'])
                for num in phone:
                    res = re.sub("[^\+.\d]"," ",num)
                node['Phone'] = res
            
            if tag.attrib['k'] in IGNORED_TAGS:
                continue
           
        if address_info != {}:
            node['address'] = address_info
    
    return node

#Cleaning the zipcodes
def clean_zipcode(zipcodes, zipcode):
    if re.match(r'^\d{5}$', zipcode):
        zipcodes.append(zipcode)
    if re.match(r'^\d{5}\:\d{5}$', zipcode):
        zips = zipcode.split(":")
        zipcodes.append(zips[0])
    return zipcodes

##Cleaning the phone numbers
def clean_phonenumber(phone_num, phones):
    if re.match(r'^[+1]?([0-1]{1})+[ -]?\(?([0-9]{3})\)?[-.●  ]?([0-9]{3})[-.●  ]?([0-9]{4})$', phones):
        phone_num.append(phones)
    #print phone_num
    if re.match(r'^\+?([0-1]{0})?[ -]?\(?([0-9]{3})\)?[-.●  ]?([0-9]{3})[-.●  ]?([0-9]{4})$',phones):
        phones = '+1-' + phones
    phone_num.append(phones)
    return phone_num
    
##Writing the data to a json file   
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            #print element
            el = shape_element(element)
            #print el
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

def test():
    # NOTE: if you are running this code on your computer, with a larger dataset, 
    # call the process_map procedure with pretty=False. The pretty=True option adds 
    # additional spaces to the output, making it significantly larger.
    data = process_map('/Users/sabihabarlaskar/Desktop/udacity-nanodegree/P3 Data wrangling/project/austin.osm', True)
    
    #pprint.pprint(data)
    
if __name__ == "__main__":
    test()